# Versão antiga que poe os posts na bdd logo no inicio, com algumas otimizações (utiliza a llm diretamente, mas nao retira duplicações)

In [1]:
from langchain_core.runnables import  RunnablePassthrough
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_ollama import ChatOllama
from neo4j import  Driver
import hashlib
from langchain_core.prompts import SystemMessagePromptTemplate, PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.docstore.document import Document
import json
import uuid

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
graph = Neo4jGraph()

C:\Users\jocke\AppData\Local\Temp\ipykernel_16516\267675202.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


# Extrair comentários de uma thread

In [3]:
import praw
import re

class RedditThreadScraper:
    def __init__(self, client_id, client_secret, user_agent):
        self.reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
        self.comment_tree = {}
        self.root_id = None

    def limpar_texto(self, text):
                """
                Cleans text from social media comments, converting citation markers and retaining all content.
                Converts '>' to '**Quoting**' and '\n\n' to '**End of Quote**' for citations, then removes
                unwanted characters and normalizes whitespace. Handles multiple citations and preserves
                all text.
                
                Args:
                    text (str): Raw text from social media comments.
                
                Returns:
                    str: Cleaned text with converted citation markers and all content preserved.
                """
                # Step 1: Split text by '\n\n' to identify citation and non-citation parts
                parts = text.strip().split('\n\n')
                converted_parts = []
                
                for part in parts:
                    part = part.strip()
                    if part.startswith('>'):
                        # Step 2: Handle citation - convert '>' to '*Quoting*'
                        citation_content = part[1:].strip()  # Remove leading '>'
                        # Add '*Quoting*' and '*End of Quote*' explicitly
                        converted_parts.append(f"**Quoting** {citation_content} **End of Quote**")
                    else:
                        # Step 3: Handle non-citation text
                        converted_parts.append(part)
                
                # Step 4: Join all parts to process as a whole
                text = '\n\n'.join(converted_parts)
                
                # Step 5: Clean the text
                # Remove unwanted characters (e.g., emojis, excessive punctuation, hashtags, mentions)
                # Preserve '*Quoting*' and '*End of Quote*' by excluding '*' from cleaning
                text = re.sub(r'[^\w\s.,!?*]', '', text)
                
                # Normalize whitespace (replace multiple spaces/tabs with single space)
                # Do this carefully to avoid collapsing '\n\n'
                lines = text.split('\n\n')
                cleaned_lines = []
                for line in lines:
                    # Normalize spaces within each line
                    cleaned_line = re.sub(r'\s+', ' ', line).strip()
                    if cleaned_line:
                        cleaned_lines.append(cleaned_line)
                
                # Step 6: Join with '\n\n' to maintain separation
                return '\n\n'.join(cleaned_lines)

    def build_comment_tree(self, thread_url):
        try:
            submission = self.reddit.submission(url=thread_url)
            titulo_op = self.limpar_texto(submission.title)
            submission.comments.replace_more(limit=0)
            self.comment_tree = {}
            self.root_id = submission.id
            self.comment_tree[self.root_id] = {
                "id": self.root_id,
                "title": titulo_op,
                "tags": [submission.link_flair_text] if submission.link_flair_text else [],  # Tags only for OP
                "text": self.limpar_texto(submission.selftext),
                "author": submission.author.name if submission.author else "Unknown",
                "parent_id": None,
                "children": []
            }
            for comment in submission.comments.list():
                if comment.author != "AutoModerator":
                    comment_id = comment.id
                    parent_id = comment.parent_id.split("_")[-1]
                    self.comment_tree[comment_id] = {
                        "id": comment_id,
                        "text": self.limpar_texto(comment.body),
                        "author": comment.author.name if comment.author else "Unknown",
                        "parent_id": parent_id,
                        "children": []  # No "tags" field for comments
                    }
                    if parent_id in self.comment_tree:
                        self.comment_tree[parent_id]["children"].append(comment_id)
            return True
        except Exception as e:
            print(f"Erro ao construir a árvore de comentários: {e}")
            self.comment_tree = {}
            self.root_id = None
            return False

    def get_comment_tree(self):
        return self.comment_tree

    def get_root_id(self):
        return self.root_id

    def print_tree(self, node_id=None, level=0):
        if not self.comment_tree:
            print("Árvore de comentários vazia.")
            return
        if node_id is None:
            node_id = self.root_id
        if node_id not in self.comment_tree:
            print(f"Nó {node_id} não encontrado na árvore.")
            return
        node = self.comment_tree[node_id]
        prefix = "➡" if level == 0 else " " * (level * 4) + "↳"
        # Only show tags for the root node (OP)
        if node_id == self.root_id:
            print(f"{prefix} {node['author']}: {node['text'][:100]} (Tags: {', '.join(node['tags'])})")
        else:
            print(f"{prefix} {node['author']}: {node['text'][:100]}")
        for child_id in node["children"]:
            self.print_tree(child_id, level + 1)

In [4]:
#from reddit_scrapper_class import *

scraper = RedditThreadScraper(
        client_id=os.getenv("REDDIT_CLIENT_ID"),
        client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
        user_agent="meu_bot_para_scraping"
    )
    
thread_url = "https://www.reddit.com/r/PoliticalDiscussion/comments/1jrwfea/do_you_think_that_there_should_be_more_or_less/?sort=new"

if scraper.build_comment_tree(thread_url):
    
    print(f"\nTítulo do OP: {scraper.comment_tree[scraper.root_id]['title']}\n")
    scraper.print_tree()




Título do OP: Do you think that there should be more or less disagreement within the Republican and Democratic parties?

➡ semiwadcutter38: Let me explain what I mean...

It seems like the the American two party system has evolved into such (Tags: US Politics)
    ↳ ryan_770: The American people are extremely divided, and typically fall pretty neatly along party lines at lea
        ↳ wheres_my_hat: It helps that one side just effectively said we arent the other guys. The republicans ditched all pr
        ↳ BluesSuedeClues: There are more independent voters than there are members of either party. I dont see that as split n
            ↳ Delanorix: Independents are a myth. Most lean towards one side and stay there.

httpswww.pewresearch.orgpolitic
                ↳ wheres_my_hat: Thats not really ground breaking. Its not like the partys flip flop on their issues enough to cause 
    ↳ BluesSuedeClues: Starting in the 1920s the Democratic Partys big tent pole issues revolved around lab

In [22]:
scraper.comment_tree

{'1jrwfea': {'id': '1jrwfea',
  'title': 'Do you think that there should be more or less disagreement within the Republican and Democratic parties?',
  'tags': ['US Politics'],
  'text': 'Let me explain what I mean... It seems like the the American two party system has evolved into such a way where each side has picked what side of an issue they fall onto and they demand almost undying loyalty of their members to everything in the platform. I think a great example of this is how the two main US political parties treat those within their own party who buck party lines regarding gun control. Of the just over 400 members of the House of Representatives that voted on an assault weapons ban in 2022, only 7 of those who voted bucked party lines, 5 Democrats and 2 Republicans to be specific. Chris Jacobs, one of the Republicans who voted in favor of the assault weapons ban, felt compelled to not seek reelection after facing backlash for voting in favor of gun control. httpswww.usatoday.comsto

In [5]:
var = scraper.comment_tree.get('mljr5wb')
print(var.get('text'))

AttributeError: 'NoneType' object has no attribute 'get'

# Exportar nodes dos comentarios para a BDD

## Para trabalhar com todos os nós 

In [5]:
# var title para inserir nos nodes "post"
topic_title = "Unknown"

# Upload nodes with MERGE to update existing nodes instead of creating duplicates
for node_id, node_data in scraper.comment_tree.items():
    if "title" in node_data:  # Original Post

        topic_title = node_data["title"]

        # Create/Update the Post node for OP
        graph.query("""
            MERGE (p:Post {id: $id})
            SET p.text = $text, p.author = $author, p.topic_title = $topic_title
            RETURN p
            """, 
            {"id": node_data["id"], "text": node_data["text"], "author": node_data["author"], "topic_title":  topic_title}
        )
        
        # Create/Update the DiscussionTopic node with tags and link it to the Post
        graph.query("""
            MERGE (dt:DiscussionTopic {title: $title})
            SET dt.tags = $tags  
            WITH dt
            MATCH (p:Post {id: $post_id})
            MERGE (dt)-[:DISCUSSES]->(p)
            """,
            {"title": node_data["title"], "post_id": node_data["id"], "tags": node_data["tags"]  }
        )
        
    else:  # Comments
        # Create/Update comment nodes as Post nodes
        graph.query("""
            MERGE (p:Post {id: $id})
            SET p.text = $text, p.author = $author, p.topic_title = $topic_title
            RETURN p
            """, 
            {"id": node_data["id"], "text": node_data["text"], "author": node_data["author"], "topic_title":  topic_title}
        )

# Create relationships RESPONDS_TO
for node_id, node_data in scraper.comment_tree.items():
    if node_data["parent_id"] and node_data["parent_id"] in scraper.comment_tree:
        graph.query("""
            MATCH (child:Post {id: $child_id}), (parent:Post {id: $parent_id})
            MERGE (child)-[:RESPONDS_TO]->(parent)
            """, 
            {"child_id": node_id, "parent_id": node_data["parent_id"]}
        )

## Para trabalhar com apenas os nós recentes

In [15]:

# Upload nodes with MERGE to update existing nodes instead of creating duplicates
nodes_to_remove = []  # Temporary list to track nodes to remove

for node_id, node_data in scraper.comment_tree.items():
    if "title" in node_data:  # Original Post
        # Check if the node exists before merging
        exists = graph.query("""
            MATCH (p:Post {id: $id})
            RETURN p
            """, 
            {"id": node_data["id"]}
        )
        
        # Create/Update the Post node for OP
        graph.query("""
            MERGE (p:Post {id: $id})
            SET p.text = $text, p.author = $author
            RETURN p
            """, 
            {"id": node_data["id"], "text": node_data["text"], "author": node_data["author"]}
        )
        
        # If it existed before the MERGE, mark for removal
        if exists and len(exists) > 0:
            nodes_to_remove.append(node_id)
            print(f"OP Node {node_id} already existed")
        
        # Create/Update the DiscussionTopic node and link it to the Post
        graph.query("""
            MERGE (dt:DiscussionTopic {title: $title})
            WITH dt
            MATCH (p:Post {id: $post_id})
            MERGE (dt)-[:DISCUSSES]->(p)
            """,
            {"title": node_data["title"], "post_id": node_data["id"]}
        )
        
    else:  # Comments
        # Check if the node exists before merging
        exists = graph.query("""
            MATCH (p:Post {id: $id})
            RETURN p
            """, 
            {"id": node_data["id"]}
        )
        
        # Create/Update comment nodes as Post nodes
        graph.query("""
            MERGE (p:Post {id: $id})
            SET p.text = $text, p.author = $author
            RETURN p
            """, 
            {"id": node_data["id"], "text": node_data["text"], "author": node_data["author"]}
        )
        
        # If it existed before the MERGE, mark for removal
        if exists and len(exists) > 0:
            nodes_to_remove.append(node_id)
            print(f"Comment Node {node_id} already existed")

# Create relationships RESPONDS_TO
for node_id, node_data in scraper.comment_tree.items():
    if node_data["parent_id"] and node_data["parent_id"] in scraper.comment_tree:
        graph.query("""
            MATCH (child:Post {id: $child_id}), (parent:Post {id: $parent_id})
            MERGE (child)-[:RESPONDS_TO]->(parent)
            """, 
            {"child_id": node_id, "parent_id": node_data["parent_id"]}
        )

# Remove nodes that already existed from scraper.comment_tree
print(f"Nodes to remove: {nodes_to_remove}")
for node_id in nodes_to_remove:
    if node_id in scraper.comment_tree:
        scraper.comment_tree.pop(node_id)

# Debug: Check final state
print(f"Final scraper.comment_tree: {scraper.comment_tree}")

OP Node 1jq1gjo already existed
Nodes to remove: ['1jq1gjo']
Final scraper.comment_tree: {'ml3p5jq': {'id': 'ml3p5jq', 'text': 'Idk, the pose is pretty common, almost a T pose or JC in the cross. The runes are pretty common, too, in any media with magic.', 'author': 'Venancio_Aires', 'parent_id': '1jq1gjo', 'children': []}, 'ml3tzdn': {'id': 'ml3tzdn', 'text': 'The description on the Death Knight Armor reads: > These knights, once Godwyn\'s personal guard, quested to find their transfigured master\'s cadaver surrogate—for the coming age of the Duskborn. I feel like the set was placed in the DLC to foreshadow Fromsoftware\'s upcoming projects, the way Dark Souls had the _Bloodborne_-coded Marvelous Chester from "a future age."', 'author': 'BadDecisions92078', 'parent_id': '1jq1gjo', 'children': []}}


In [7]:
scraper.comment_tree

{'1k2ts52': {'id': '1k2ts52',
  'title': 'What are your thoughts on the UKs national rebirth party?',
  'tags': ['European Politics'],
  'text': 'httpsnationalrebirthparty.org.ukAgenda\n\nThe National Rebirth Party NRP is a political party in the United Kingdom that emerged in the early 21st century. It is generally characterized by its nationalist and rightwing populist ideologies. The party advocates for policies that emphasize British sovereignty, immigration control, and a return to traditional values.\n\nThe NRP typically focuses on issues such as promoting the interests of British citizens, criticizing the influence of the European Union, and opposing multiculturalism, which they believe undermines national identity. The party has also expressed concerns about social issues, advocating for a strong law and order approach.\n\nWhile the NRP aims to resonate with voters who feel disillusioned with mainstream politics, it has faced criticism for its views, which some label as extremi

# Recuperação de contexto e criação de prompt

In [6]:
# Pedir a contextualizão de argumentos pode levar a tempos de processamento maiores (de 3 passou para 6 segundos)
system_prompt = SystemMessagePromptTemplate.from_template("""
You are a system that extracts political arguments and their associated motivations from input text. Your goal is to return a dataset of clearly defined [Argument] and [Motivation] sections in **JSON format**. 

You will be given a political discussion thread with multiple comments. These comments are presented together as input, and may include original posts and replies. You should extract arguments and motivations based on the full context of the thread — not just the last comment.

                                                          
Each [Argument] must contain a single contextualized and coherent political claim or position. Each [Motivation] must explain a reason behind the argument and be associated with one or more of the following Max-Neef categories:
Subsistence, Protection, Affection, Understanding, Participation, Leisure, Creativity, Identity, Freedom.

You must preserve a clear link between each Argument and its corresponding Motivations by nesting motivations under their respective argument.

Here are a few examples of the expected structure:

[
  {{
    "argument": "Tax policy should be expanded to support economic growth",
    "motivations": [
      {{
        "description": "Wants economic stability for all",
        "max_neef_category": ["Subsistence"]
      }}
    ]
  }},
  {{
    "argument": "Climate change needs tighter restrictions to safeguard future generations",
    "motivations": [
      {{
        "description": "Wants to secure the future",
        "max_neef_category": ["Protection"]
      }},
      {{
        "description": "Desires collective action",
        "max_neef_category": ["Participation"]
      }}
    ]
  }},
  {{
    "argument": "Healthcare policies should focus on advancing social equity",
    "motivations": [
      {{
        "description": "Hopes to support the vulnerable",
        "max_neef_category": ["Affection"]
      }}
    ]
  }},
  {{
    "argument": "Gun control needs tighter restrictions to protect public safety",
    "motivations": [
      {{
        "description": "Fears threats to safety",
        "max_neef_category": ["Protection"]
      }}
    ]
  }},
  {{
    "argument": "Arts funding should prioritize fostering cultural development",
    "motivations": [
      {{
        "description": "Enjoys cultural enrichment",
        "max_neef_category": ["Leisure"]
      }},
      {{
        "description": "Wants innovative solutions",
        "max_neef_category": ["Creativity", "Understanding"]
      }}
    ]
  }}
]

Using this JSON format, extract arguments and motivations from the input text. Each argument must be politically relevant and distinct. Motivations must be logically aligned with the argument.
""")

human_prompt = HumanMessagePromptTemplate.from_template("""
You are given a sequence of political comments from a discussion thread. Use the entire thread as context.

Make sure to group Arguments with their respecting Motivations. Use the format shown above to extract the relevant information from the following input:

Thread:
                                                        
{input}

Extract arguments and motivations **only from the last comment**, and group each argument with its respective motivations.

                                                        
Make sure **each Argument has one or more associated Motivations**, and that the response uses the correct format. Do not include any commentary or explanations.

""")

chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])


## Funções para recolher contexto

In [7]:

summarizer = ChatOllama(model="llama3.1")

# Função para buscar os pais de um post (até 3 níveis)
def get_parent_posts(post_id: str, max_levels: int = 3):
    query = f"""
    MATCH (child:Post {{id: $post_id}})-[:RESPONDS_TO*1..{max_levels}]->(parent:Post)
    WHERE toLower(parent.text) <> '[removed]'
    RETURN parent.text AS text, parent.id AS id
    ORDER BY size(parent.text) DESC
    """
    results = graph.query(query, params={"post_id": post_id})
    return [{"id": row["id"], "text": row["text"]} for row in results]

# Função para buscar o texto do post atual
def get_post_text(post_id: str):
    query = """
    MATCH (p:Post {id: $post_id})
    RETURN p.text AS text
    """
    result = graph.query(query, params={"post_id": post_id})
    return result[0]["text"] if result else None

# Função para sumarizar texto com LLM
def resumir_texto(texto: str) -> str:
    prompt = f"""
    Summarize the following post in a concise and informative way. If you refer to the author, make sure you refer to them as 'an author of previous posts'. Only keep what is essential to understand the point made, and return only the summary:

    \"\"\"{texto}\"\"\"
    """
    return summarizer.invoke(prompt).content

# Função principal para construir prompt com contexto sumarizado
def get_context(post_id: str, max_pais: int = 3):
    pais = get_parent_posts(post_id, max_levels=max_pais)
    post_filho = get_post_text(post_id)
    

    if not post_filho:
        raise ValueError("Post filho não encontrado.")

    contexto_resumido = []
    for p in pais[:max_pais]:
        resumo = resumir_texto(p["text"])
        contexto_resumido.append(f"*Context:* {resumo}")
        

    contexto_final = "\n".join(contexto_resumido)

    
    return  contexto_final
    




In [8]:
# Exemplo de utilização

post_id_input = "mo2mhmn"  
context = get_context(post_id_input)


In [9]:
print(context)

*Context:* The National Rebirth Party (NRP) is a UK political party with nationalist and right-wing populist ideologies. They prioritize British sovereignty, immigration control, and traditional values. Their views have faced criticism as extremist or xenophobic, but they represent a segment of voters seeking a radical change from mainstream politics.


# Preparar loop de processamento

In [17]:
def json_para_graph_document(data: dict, input_text: str) -> GraphDocument:
    graph_documents = []
    
    nodes = []
    relationships = []
    
    for entry in data.get("arguments", []):
        # Cria o nó do argumento
        argument_id = str(uuid.uuid4())
        argument_node = Node(
            id=argument_id,
            type="Argument",
            properties={"description": entry["argument"]}
        )
        nodes.append(argument_node)
        
        for motivation in entry.get("motivations", []):
            # Cria o nó da motivação
            motivation_id = str(uuid.uuid4())
            motivation_node = Node(
                id=motivation_id,
                type="Motivation",
                properties={
                    "description": motivation["description"],
                    "max_neef_category": motivation["max_neef_category"]
                }
            )
            nodes.append(motivation_node)
            
            # Cria a relação entre motivação e argumento
            relationship = Relationship(
                source=motivation_node,
                target=argument_node,
                type="supports",
                properties={}
            )

            relationships.append(relationship)

    # Documento de origem (comentário, post, etc.)
    doc = Document(page_content=input_text)

    graph_doc = GraphDocument(
        nodes=nodes,
        relationships=relationships,
        source=doc
    )
    
    graph_documents.append(graph_doc)
    return graph_documents



In [11]:
llm = ChatOllama(model="llama3.1", temperature=0, format="json")

def extract_nodes(input_text):
    messages = chat_prompt.format_messages(input=input_text)
    response = llm.invoke(messages)
    
    return json.loads(response.content)   



In [ ]:

comment_tree = scraper.comment_tree

In [32]:
# temp

comment_tree.values()

dict_values([{'id': '1k2ts52', 'title': 'What are your thoughts on the UKs national rebirth party?', 'tags': ['European Politics'], 'text': 'httpsnationalrebirthparty.org.ukAgenda\n\nThe National Rebirth Party NRP is a political party in the United Kingdom that emerged in the early 21st century. It is generally characterized by its nationalist and rightwing populist ideologies. The party advocates for policies that emphasize British sovereignty, immigration control, and a return to traditional values.\n\nThe NRP typically focuses on issues such as promoting the interests of British citizens, criticizing the influence of the European Union, and opposing multiculturalism, which they believe undermines national identity. The party has also expressed concerns about social issues, advocating for a strong law and order approach.\n\nWhile the NRP aims to resonate with voters who feel disillusioned with mainstream politics, it has faced criticism for its views, which some label as extremist or

In [ ]:
graph_documents = []
n_comments = len(comment_tree.values())
count = 1
for comment_info in comment_tree.values():
    print(f"Processing comment {count}/{n_comments}")
    count += 1

    comment = comment_info.get("text")
    context = get_context(comment_info.get("id"))
    input = context + "\n" +comment

    raw_format = extract_nodes(input)
    temp_doc = json_para_graph_document(raw_format, comment)
    
    temp_doc[0].source.metadata['post_id'] = comment_info.get("id") 

    graph_documents.extend(temp_doc)


Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment
Processing a new comment


In [19]:
for doc in graph_documents:
    print(doc.relationships)

[Relationship(source=Node(id='07565959-087f-49e7-958b-a577984995a4', type='Motivation', properties={'description': 'Wants to avoid binary thinking', 'max_neef_category': ['Understanding']}), target=Node(id='d41907fa-6b62-432f-aeaf-f90e06863b9f', type='Argument', properties={'description': 'Politics should offer more nuanced choices beyond two extreme options'}), type='supports', properties={}), Relationship(source=Node(id='31f67a33-c453-4d9c-a260-0e0816f87089', type='Motivation', properties={'description': 'Desires more flexibility in political decisions', 'max_neef_category': ['Participation', 'Freedom']}), target=Node(id='d41907fa-6b62-432f-aeaf-f90e06863b9f', type='Argument', properties={'description': 'Politics should offer more nuanced choices beyond two extreme options'}), type='supports', properties={}), Relationship(source=Node(id='0a24d941-35f5-4bb5-8074-a2eb918647fe', type='Motivation', properties={'description': 'Wants to promote critical thinking and debate within parties',

# Exportar nodes e relações

In [20]:
# adicionar nodes origem

# Upload do grafo com argumentos e motivações
graph.add_graph_documents(graph_documents=graph_documents, include_source=False)

# Criar ou obter o nó do tópico e conectar posts a ele
for doc in graph_documents:
    post_id = doc.source.metadata.get('post_id')
    # Conectar argumentos ao post
    for node in doc.nodes:
        if "Argument" in node.type:
            graph.query(
                """
                MATCH (p:Post {id: $post_id}), (n:Argument {id: $node_id})
                MERGE (p)-[:STATED]->(n)
                """,
                {"post_id": post_id, "node_id": node.id}
            )





# Vetores

In [ ]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    index_name="postEmbeddingIndex",
    search_type="hybrid",
    node_label="Post",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [15]:
driver = GraphDatabase.driver(
        uri = os.getenv("NEO4J_URI"),
        auth = (os.getenv("NEO4J_USERNAME"),
                os.getenv("NEO4J_PASSWORD")))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

Fulltext index created successfully.


In [18]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting information from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = prompt | llm.with_structured_output(Entities)

In [29]:
entity_chain.invoke({"question":"Minimum wage?"})

Entities(names=['Minimum wage'])

In [31]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question":question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL (node){
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [32]:
graph_retriever("Minimum Wage")

''

# Debugging

In [31]:


llm = OllamaFunctions(model="deepseek-r1:8b", temperature=0.1, format="json")

In [34]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

graph_transformer = LLMGraphTransformer(llm=llm, prompt=chat_prompt, allowed_nodes=allowed_nodes, node_properties=["description"], strict_mode=False)

In [37]:
text = "Alice knows Bob. Bob works at xAI."
document = Document(page_content=text, metadata={"source": "example"})


In [39]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

graph_transformer = LLMGraphTransformer(llm=llm)
graph_documents = graph_transformer.convert_to_graph_documents([document])

ValueError: Failed to parse a response from deepseek-r1:8b output: {


}

  


  


  


  


  


  


   


   


   


 

In [12]:

text_comment = "Starting in the 1920's the Democratic Party's big tent pole issues revolved around labor rights and support of unions. By the 1960's, successful legislation outlining workers rights, occupational safety standards, a minimum wage and environmental working conditions, had increasingly made that focus feel less urgent, letting civil rights become the predominant issue for the party. Since then, the party has become a coalition of varied interests (climate change, reproductive rights, LGBTQ rights, etc.) and lacks a unifying issue to coalesce the party. This leaves them squabbling over the direction of the party, in every election cycle, with the centrist establishment usually holding sway, while the more progressive members become increasingly dissatisfied and uncooperative with party goals. The Republican Party has changed more from two single inflection points, then through slow modulation over time. The first being Nixon's Southern Strategy, which was to employ racist dog-whistle rhetoric to lure traditional Democratic voters in the South, the Dixiecrats, into the Republican fold. Around the same time, they adopted an anti-abortion stance (which had previously not been a big political issue) in order to lure traditionally Democratic Catholic voters into the GOP (having the unusual side result of Protestant Christians adopting their views on an issue from their political leaders, rather than the other way around). The second one was Trump. He has remade the Republican Party into the Trump party, and he rules over it in an openly authoritarian manner. The GOP has become more authoritarian in response and hence more cohesive in their voting habits, as demonstrating loyalty to Donald Trump is more important than maintaining a consistent political stance on any given issue. You see this contradiction with Trump spending a decade demeaning and rejecting electric vehicles, then doing a sales advert on the White House South Lawn, pushing electric vehicles. His supporters go along with this, because authoritarian structures demand conformity. Given the very different ideological structure of these two parties, it becomes difficult for them to even have a dialog about policies for the US. One side is inclined to argue, bicker and fight, but also feels obligated to let even the smallest, most marginalized voices be heard (even if they then ignore those voices). The other side is beholden to a single authority sitting at the peak of a top-down power structure. Their very understanding of who has power and how to wield it, is diametrically opposed. This makes it difficult for them to even agree on a framework for discussion, let alone a consensus on policy. **TL/DR:** Republicans value conformity and authority, making it more likely they fall in line to the leaders goals, but also more likely to support bad policy, just to demonstrate loyalty. They also embrace one large issue as uniting (currently it's immigration). Democrats value inclusion and consensus, making their internal functions more fractious, messy and less likely to arrive at a unifying policy decision. They were more successful when they agreed on one issue being more important than the myriad issues of their voters (as they did with Labor and then Civil rights), but less inclusive. Honestly, the Republican approach is more efficient, but also more likely to harm marginalized groups and damage the function of government (which is supposed to be a consensus, not just obedience). But the Democrats approach is very difficult to corral into sensible policy goals. The dems would be wise to choose a tent pole issue, like healthcare or wealth inequality, even if it costs them some of their fringe elements."
docs = [Document(page_content=text_comment)]  # Create a list with a single Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=280, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [13]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph_documents




ValueError: Failed to parse a response from deepseek-r1:8b output: {}

   


  


  


  


  


  


  








  








  








  










In [23]:
# Ver N de argumentos e motivacoes
arg = 0
mot = 0
for doc in graph_documents:
        for node in doc.nodes:
                print(node)      
                if node.type == 'Argument': arg += 1 
                else: mot += 1

print("arg = ", arg)
print("mot = ", mot)


id='Argument_31cfba73' type='Argument' properties={'description': 'The pose is pretty common, almost a T pose or JC in the cross.'}
id='Motivation_770a02a6' type='Motivation' properties={'description': 'pretty common', 'max_neef_category': 'Subsistence'}
id='Argument_c4c38397' type='Argument' properties={'description': 'in any media with magic.'}
id='Motivation_770a02a6' type='Motivation' properties={'description': 'pretty common', 'max_neef_category': 'Protection'}
arg =  2
mot =  2


In [40]:
comment_tree = scraper.comment_tree
print(comment_tree.get(scraper.root_id).get("title"))

Whats the recommended lv for the dlc?
